# Tabulation of Fractional Cover data within shapefile polygons

**What does this notebook do?**

This notebook is a pilot collaboration between Geoscience Australia and Australian Bureau of Statistics. The purpose of the notebook is to use a shapefile polygon boundaries to load fractional cover dataset, complete zonal statistics and tabulate the results.

**Requirements**

You need to run the following commands from the command line prior to launching jupyter notebooks from the same terminal so that the required libraries and paths are set:

`module use /g/data/v10/public/modules/modulefiles`

`module load dea/20181213`


**Background**

Data from the Landsat 5,7 and 8 satellite missions are accessible through Digital Earth Australia (DEA). The code snippets in this notebook will let you retrieve and plot the Fractional Cover (FC25) data stored in DEA.


**How to use this notebook**

A basic understanding of any programming language is desirable but one doesn't have to be an expert Python programmer to manipulate the code to get and display the data.This doc applies to the following Landsat satellites, Fractional Cover bands and the WOfS dataset:

- Landsat 5
- Landsat 7
- Landsat 8
- PV - Photosythetic vegetation
- NPV - Non-Photosythetic vegetation
- BS - Bare Soil
- UE - Unmixing Error
- Water Observations from Space (WOFs)
- WOfS Feature Layer (WOFL)

**Bugs still to fix**

- Memory errors for extra large polygons - AH & ET


**Errors or bugs**

If you find an error or bug in this notebook, please contact erin.telfer@ga.gov.au.


## 1. Import Libraries

In [4]:
%matplotlib inline

from datetime import time, datetime
import os.path

from matplotlib import pyplot as plt
import pandas as pd
import numpy
import csv
import xarray as xr
import rasterio
import rasterio.features
import fiona
from datetime import datetime
import dask
from dask.delayed import delayed
from dask.distributed import LocalCluster, Client
import tempfile

import datacube
from datacube import Datacube
from datacube.virtual import construct, construct_from_yaml
from datacube.ui.task_app import year_splitter
from datacube.utils.geometry import CRS

print(f"Cell finished at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

Cell finished at 2019-03-22 11:27:01


## 2. Define directories and years of interest

In [5]:
#Set folder  and SA2 shapefile locations
shapefile_path = '../input/SA2_2016_AUST.shp'
output_path = '/g/data/r78/ext547/abs/output/'

In [6]:
#Specify time years of interest
start_year, end_year = 2011, 2016
time_range = (str(start_year), str(end_year))
print(time_range)

('2011', '2016')


## 3. Set up a local dask cluster
Some calculations take more memory than is available on a system.  By breaking the data up into chunks, we can chain a sequence of operations together, and work on the data a small piece at a time.

This lets several processes work at the same time, and manage total memory usage for the calculations.

In more advanced setups, we can distribute the work across multiple computers, using all of their memory and CPU power.

In [7]:
#Set up dask cluster
n_workers = 7
threads_per_worker=1
mem_per_worker = 8e9  # 8e9 is 8GB (8,000,000,000 bytes)

chunk_size = {'time': 1, 'x': 2000, 'y': 2000}

In [8]:
cluster = LocalCluster(local_dir=tempfile.gettempdir(), 
                       n_workers=n_workers, 
                       threads_per_worker=threads_per_worker,
                       memory_limit=mem_per_worker)
client = Client(cluster)
dask.config.set(get=client.get)
client

Client Scheduler: tcp://127.0.0.1:34997 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 7 Cores: 7 Memory: 56.00 GB


We also get a dashboard to see how the system is running, by clicking the link above after the cell has been run.

## 4. Connect to the Datacube 

In [9]:
dc = Datacube()
print(f"Cell finished at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

Cell finished at 2019-03-22 11:27:09


## 5. Construct the virtual product

In [10]:
#Remove Landsat 7 scenes with the Scan Line Correction (SLC) missing data
LS7_BROKEN_DATE = datetime(2003, 5, 31)
is_pre_slc_failure = lambda dataset: dataset.center_time < LS7_BROKEN_DATE
print(f"Cell finished at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

Cell finished at 2019-03-22 11:27:09


In [11]:
#Create function to ensure wofls in correct format
def wofls_fuser(dest, src):
    where_nodata = (src & 1) == 0
    numpy.copyto(dest, src, where=where_nodata)
    return dest
print(f"Cell finished at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

Cell finished at 2019-03-22 11:27:09


In [12]:
#Create virtual product so that datacube data can be loaded effectively within memory
fc_and_water_yaml = """
        juxtapose:
          - collate:
              - transform: apply_mask
                mask_measurement_name: pixelquality
                preserve_dtype: false
                input:
                    juxtapose:
                      - product: ls5_fc_albers
                        group_by: solar_day
                        measurements: [PV, NPV, BS]
                      - transform: make_mask
                        input:
                            product: ls5_pq_albers
                            group_by: solar_day
                            fuse_func: datacube.helpers.ga_pq_fuser
                        flags:
                            ga_good_pixel: true
                        mask_measurement_name: pixelquality
              - transform: apply_mask
                mask_measurement_name: pixelquality
                preserve_dtype: false
                input:
                    juxtapose:
                      - product: ls7_fc_albers
                        group_by: solar_day
                        measurements: [PV, NPV, BS]
                        # dataset_predicate: __main__.is_pre_slc_failure
                      - transform: make_mask
                        input:
                            product: ls7_pq_albers
                            group_by: solar_day
                            fuse_func: datacube.helpers.ga_pq_fuser
                        flags:
                            ga_good_pixel: true
                        mask_measurement_name: pixelquality
              - transform: apply_mask
                mask_measurement_name: pixelquality
                preserve_dtype: false
                input:
                    juxtapose:
                      - product: ls8_fc_albers
                        group_by: solar_day
                        measurements: [PV, NPV, BS]
                      - transform: make_mask
                        input:
                            product: ls8_pq_albers
                            group_by: solar_day
                            fuse_func: datacube.helpers.ga_pq_fuser
                        flags:
                            ga_good_pixel: true
                        mask_measurement_name: pixelquality
          - transform: make_mask
            input:
                product: wofs_albers
                group_by: solar_day
                fuse_func: __main__.wofls_fuser
            flags:
                wet: true
            mask_measurement_name: water
"""
fc_and_water = construct_from_yaml(fc_and_water_yaml)

## 6. Set up functions

In [13]:
def geometry_mask(geoms, geobox, all_touched=False, invert=False, chunks=None):
    """
    Create a mask from shapes.

    By default, mask is intended for use as a
    numpy mask, where pixels that overlap shapes are False.
    :param list[Geometry] geoms: geometries to be rasterized
    :param datacube.utils.GeoBox geobox:
    :param bool all_touched: If True, all pixels touched by geometries will be burned in. If
                             false, only pixels whose center is within the polygon or that
                             are selected by Bresenham's line algorithm will be burned in.
    :param bool invert: If True, mask will be True for pixels that overlap shapes.
    """
    data = rasterio.features.geometry_mask([geom.to_crs(geobox.crs) for geom in geoms],
                                           out_shape=geobox.shape,
                                           transform=geobox.affine,
                                           all_touched=all_touched,
                                           invert=invert)
    if chunks is not None:
        data = dask.array.from_array(data, chunks=tuple(chunks[d] for d in geobox.dims))
        
    coords = [xr.DataArray(data=coord.values, name=dim, dims=[dim], attrs={'units': coord.units}) 
              for dim, coord in geobox.coords.items()]
    return xr.DataArray(data, coords=coords)
print(f"Cell finished at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

Cell finished at 2019-03-22 11:27:10


In [14]:
def get_shapes(shape_file):
    """
    Extract spatial inforamtion from polygons within shapefile
    """
    with fiona.open(shape_file) as shapes:
        crs = datacube.utils.geometry.CRS(shapes.crs_wkt)
        for shape in shapes:
            if shape['geometry'] is None:
                continue
            geom = datacube.utils.geometry.Geometry(shape['geometry'], crs=crs)
            geom = geom.to_crs(CRS('EPSG:3577'))
            yield geom, shape['properties']
print(f"Cell finished at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

Cell finished at 2019-03-22 11:27:11


In [15]:
def fc_and_water_summary(data, mask_int):
    """
    Calculate the percentage and the area of each FC component, 
    water, and unclassified data within each SA2 boundary. 
    """    
    # Where there is water, set the FC bands to 0%
    fc = data[['PV', 'NPV', 'BS']].where(data.water==0, other=0)
    fc['water'] = data.water.where(numpy.isfinite(fc['BS'])) * numpy.float32(100)
    fc = fc.apply(lambda data_array: data_array.clip(0, 100).where(numpy.isfinite(data_array)))
    
    # Flatten to a monthly mean
    fc = fc.groupby(fc.time.astype('datetime64[M]'), squeeze=False).mean(dim='time', skipna=True)
    # Calculate percentage of cover based on area of mask
    percentage = fc.sum(dim=['x','y']) * (100 / mask_int)
    
    for da in percentage.data_vars.values():
        da.attrs['units'] = '%'

    fc_unclass = percentage.to_array('variable').sum(dim='variable')
    fc_unclass = 100 - float(fc_unclass.values)
    print(f"   PV = {int(percentage.PV.values)}%, NPV = {int(percentage.NPV.values)}%, BS = {int(percentage.BS.values)}%, water = {int(percentage.water.values)}%, unclassified = {int(fc_unclass)}%")
    
    fc['unclassified'] = ('time', numpy.repeat(fc_unclass,fc.time.size)) 
    percentage['unclassified'] = ('time', numpy.repeat(fc_unclass,fc.time.size)) 
        
    pixel_area_in_metres2 = 25 * 25
    m2_to_km2 = (1 / 1_000_000)
    percent_to_fraction = (1 / 100)
    
    area = (fc * (pixel_area_in_metres2 * m2_to_km2 * percent_to_fraction)).sum(dim=['x','y'])
    area = area.rename({'BS': 'BS_area', 
                        'PV': 'PV_area', 
                        'NPV': 'NPV_area', 
                        'water': 'water_area', 
                        'unclassified':'unclassified_area'})
    
    for da in area.data_vars.values():
        da.attrs['units'] = 'km^2'
    fc = percentage.merge(area)
    return fc

In [16]:
def plot_stacked(ds, sa2_id,plot_title='title', show=True):
    """
    Create and save a stacked plot to visualise FC components
    """
    if not show:
        plt.ioff()
        
    fig,ax = plt.subplots(figsize=(10,5))
    ax.stackplot(ds.dropna(dim='time').time.data, 
                 ds.dropna(dim='time').PV,
                 ds.dropna(dim='time').NPV, 
                 ds.dropna(dim='time').BS, 
                 ds.dropna(dim='time').water, 
                 ds.dropna(dim='time').unclassified, 
                 colors = ['darkolivegreen','olive','tan','darkblue','red'], 
                 labels=['PV','NPV','BS','Water','Unclassified',])
    plt.legend(loc='upper center', ncol = 5)
    plt.title(f'FC components: SA2 ID {sa2_id}', size=12)
    plt.ylabel('Percentage (%)', size=12) #Set Y label
    plt.xlabel('Date', size=12) #Set X label
    
    plt.savefig(f'{output_path}/{sa2_id}_{plot_title}.png');
    plt.close(fig)
    
    # Turn interactive back on
    if not show:
        plt.show()
print(f"Cell finished at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

Cell finished at 2019-03-22 11:27:13


In [17]:
def month_splitter(start_year, end_year_inclusive):
    """ 
    Split specified years into months 
    """
    yield from (str(p) for p in pd.period_range(start=pd.Period(start_year).start_time, 
                               end=pd.Period(end_year_inclusive).end_time, 
                               freq='M'))

In [18]:
def output_csv(input_ds, sa2_id,monthly_or_annual='frequency'):
    """
    Save tabulated data into a csv
    """
    input_ds = input_ds.to_dataframe()
    input_ds.insert(0,'sa2',sa2_id)
    input_ds.to_csv(f"{output_path}/tabulate_{monthly_or_annual}_FC.csv",mode='a',header=False)  

## 7. Set up the query
For each year and polygon query the product, apply the geometry mask and compute the fractional cover stats

Using `client.compute()` lets us use the monthly results in calculating the annual results at the same time.

In [20]:
#Obtain spatial information from shapefile
shape_file = os.path.expanduser(f'{shapefile_path}')
shapes = list(get_shapes(shape_file))

# #Specify a particular SA2 boundary, if required
# shapes = [(g,p) for g, p in shapes if str(p['SA2_MAIN16']) == '312011340']
shapes = [(g,p) for g, p in shapes if str(p['AREASQKM16']) < '15_000']


print(f"Cell finished at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

Cell finished at 2019-03-22 11:30:36


If we have enough resources, we can start the query and calculation of the next year's data while the previous is still being calculated. `by_slice=False` will be faster, but use more memory.

For larger areas `by_slice` will need to be `True`, so that the compute cluster does not become overwhelmed.  

If you get the error:
> `distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting`

then you will need to set `by_slice=True`

In [21]:
by_slice=True

In [22]:
def process_area(geometry, sa2_id, time_range):
    monthly_values = []
    annual_values = []
    mask = None

    for sub_time_range in month_splitter(time_range[0], time_range[-1]):
        print(sub_time_range)
        try:
            data = fc_and_water.load(dc, dask_chunks=chunk_size, 
                                     time=sub_time_range, 
                                     geopolygon=geometry)
        except ValueError:
            print(f'    No data for {sub_time_range} , skipping...')
            continue

        if mask is None:
            mask = geometry_mask([geometry], data.geobox, invert=True, chunks=data.chunks)
            mask_int = mask * 1
            mask_int = mask_int.sum() * 100
        data = data.where(mask)
        monthly_data = fc_and_water_summary(data, mask_int)
        monthly_data = client.compute(monthly_data, sync=by_slice)
        monthly_values.append(monthly_data)
        
    if not by_slice:
        print("  all years queried, hard load data")
        monthly_values = client.gather(monthly_values)

    monthly_values = xr.concat(monthly_values, dim='time')
    monthly_values = monthly_values.where(monthly_values['unclassified'] < 10).dropna(dim='time')
    annual_values = monthly_values.resample(time='1YS').mean(dim='time', skipna=True)
    print(f"Calculation complete for annual values")
    
    plot_stacked(monthly_values, sa2_id, plot_title='monthly_plot_wofs',show=False)
    plot_stacked(annual_values, sa2_id, plot_title='annual_plot_wofs', show=False)
    
    print("All data loaded, save to csv")
    output_csv(monthly_values, sa2_id,monthly_or_annual='monthly')
    output_csv(annual_values, sa2_id,monthly_or_annual='annual')
    
    print(f"SA2 {sa2_id} done")

## 8. Process query

In [23]:
#Create empty CSV with specified headings
header = ['Date','sa2_id','PV','NPV','BS','WOFL',
          'UNCLASSIFIED','PV_AREA_SQKM_ALBERS3577','NPV_AREA_SQKM_ALBERS3577',
          'BS_AREA_SQKM_ALBERS3577','WOFL_AREA_SQKM_ALBERS3577','UNCLASSIFIED_AREA_SQKM_ALBERS3577']

with open(f"{output_path}/tabulate_annual_FC.csv","w") as outcsv: #create csv to save output and add header text
    writer = csv.writer(outcsv)
    writer.writerow(header)
    
with open(f"{output_path}/tabulate_monthly_FC.csv","w") as outcsv: #create csv to save output and add header text
    writer = csv.writer(outcsv)
    writer.writerow(header)

In [ ]:
#Run tabulation script
for geometry, properties in shapes:
    sa2_id = str(properties['SA2_MAIN16'])
    print(f"SA2 ID: {sa2_id}, size: {properties['AREASQKM16']}km^2, time: {time_range}")
    
    try:
          process_area(geometry, sa2_id, time_range)
    except Exception as e:
          print(f"Could not process {sa2_id}: {e}")
          client.restart()

SA2 ID: 101021010, size: 13.0034km^2, time: ('2011', '2016')
2011-01
   PV = 46%, NPV = 32%, BS = 19%, water = 0%, unclassified = 1%
2011-02
   PV = 0%, NPV = 0%, BS = 0%, water = 0%, unclassified = 100%
2011-03
   PV = 48%, NPV = 30%, BS = 19%, water = 0%, unclassified = 1%
2011-04
   PV = 42%, NPV = 27%, BS = 20%, water = 0%, unclassified = 7%
2011-05
   PV = 34%, NPV = 26%, BS = 20%, water = 0%, unclassified = 18%
2011-06
   PV = 33%, NPV = 25%, BS = 17%, water = 0%, unclassified = 23%
2011-07
   PV = 39%, NPV = 34%, BS = 22%, water = 0%, unclassified = 3%
2011-08
   PV = 27%, NPV = 25%, BS = 27%, water = 0%, unclassified = 19%
2011-09
   PV = 36%, NPV = 38%, BS = 23%, water = 0%, unclassified = 1%
2011-10
   PV = 33%, NPV = 32%, BS = 17%, water = 0%, unclassified = 16%
2011-11
   PV = 6%, NPV = 4%, BS = 7%, water = 0%, unclassified = 81%
2011-12
   PV = 26%, NPV = 23%, BS = 15%, water = 0%, unclassified = 34%
2012-01
   PV = 37%, NPV = 34%, BS = 25%, water = 0%, unclassified = 1%
2

2014-04
   PV = 44%, NPV = 20%, BS = 33%, water = 0%, unclassified = 1%
2014-05
   PV = 43%, NPV = 25%, BS = 29%, water = 0%, unclassified = 1%
2014-06
   PV = 29%, NPV = 16%, BS = 28%, water = 0%, unclassified = 25%
2014-07
   PV = 38%, NPV = 26%, BS = 32%, water = 0%, unclassified = 2%
2014-08
   PV = 36%, NPV = 28%, BS = 33%, water = 0%, unclassified = 1%
2014-09
   PV = 40%, NPV = 25%, BS = 32%, water = 0%, unclassified = 1%
2014-10
   PV = 44%, NPV = 24%, BS = 26%, water = 0%, unclassified = 4%
2014-11
   PV = 10%, NPV = 6%, BS = 6%, water = 0%, unclassified = 76%
2014-12
   PV = 40%, NPV = 32%, BS = 26%, water = 0%, unclassified = 1%
2015-01
   PV = 39%, NPV = 32%, BS = 26%, water = 0%, unclassified = 1%
2015-02
   PV = 0%, NPV = 0%, BS = 0%, water = 0%, unclassified = 100%
2015-03
   PV = 35%, NPV = 33%, BS = 29%, water = 0%, unclassified = 1%
2015-04
   PV = 35%, NPV = 31%, BS = 31%, water = 0%, unclassified = 1%
2015-05
   PV = 33%, NPV = 27%, BS = 26%, water = 0%, unclassifie

2011-07
   PV = 63%, NPV = 17%, BS = 5%, water = 0%, unclassified = 13%
2011-08
   PV = 60%, NPV = 16%, BS = 12%, water = 0%, unclassified = 10%
2011-09
   PV = 71%, NPV = 22%, BS = 3%, water = 1%, unclassified = 1%
2011-10
   PV = 64%, NPV = 16%, BS = 5%, water = 1%, unclassified = 12%
2011-11
   PV = 45%, NPV = 13%, BS = 8%, water = 1%, unclassified = 31%
2011-12
   PV = 29%, NPV = 6%, BS = 3%, water = 0%, unclassified = 59%
2012-01
   PV = 74%, NPV = 19%, BS = 2%, water = 1%, unclassified = 1%
2012-02
   PV = 43%, NPV = 9%, BS = 4%, water = 1%, unclassified = 41%
2012-03
   PV = 64%, NPV = 17%, BS = 2%, water = 1%, unclassified = 13%
2012-04
   PV = 10%, NPV = 1%, BS = 5%, water = 0%, unclassified = 82%
2012-05
   PV = 65%, NPV = 14%, BS = 7%, water = 0%, unclassified = 12%
2012-06
   PV = 48%, NPV = 9%, BS = 9%, water = 0%, unclassified = 31%
2012-07
   PV = 67%, NPV = 15%, BS = 5%, water = 0%, unclassified = 10%
2012-08
   PV = 58%, NPV = 17%, BS = 2%, water = 0%, unclassified = 2

2015-01
   PV = 61%, NPV = 14%, BS = 8%, water = 0%, unclassified = 15%
2015-02
   PV = 71%, NPV = 16%, BS = 10%, water = 0%, unclassified = 1%
2015-03
   PV = 49%, NPV = 12%, BS = 9%, water = 0%, unclassified = 28%
2015-04
   PV = 8%, NPV = 1%, BS = 5%, water = 0%, unclassified = 84%
2015-05
   PV = 60%, NPV = 16%, BS = 7%, water = 0%, unclassified = 15%
2015-06
   PV = 68%, NPV = 15%, BS = 13%, water = 0%, unclassified = 2%
2015-07
   PV = 66%, NPV = 19%, BS = 12%, water = 0%, unclassified = 2%
2015-08
   PV = 65%, NPV = 19%, BS = 13%, water = 0%, unclassified = 1%
2015-09
   PV = 63%, NPV = 21%, BS = 13%, water = 0%, unclassified = 1%
2015-10
   PV = 62%, NPV = 16%, BS = 11%, water = 0%, unclassified = 9%
2015-11
   PV = 60%, NPV = 15%, BS = 9%, water = 0%, unclassified = 14%
2015-12
   PV = 66%, NPV = 19%, BS = 12%, water = 0%, unclassified = 1%
2016-01
   PV = 65%, NPV = 17%, BS = 15%, water = 0%, unclassified = 0%
2016-02
   PV = 67%, NPV = 18%, BS = 12%, water = 0%, unclassified

2012-04
    No data for 2012-04 , skipping...
2012-05
   PV = 56%, NPV = 20%, BS = 15%, water = 0%, unclassified = 6%
2012-06
   PV = 62%, NPV = 20%, BS = 14%, water = 0%, unclassified = 2%
2012-07
   PV = 29%, NPV = 5%, BS = 10%, water = 0%, unclassified = 53%
2012-08
   PV = 55%, NPV = 22%, BS = 17%, water = 2%, unclassified = 1%
2012-09
   PV = 54%, NPV = 24%, BS = 15%, water = 3%, unclassified = 1%
2012-10
   PV = 48%, NPV = 21%, BS = 12%, water = 3%, unclassified = 14%
2012-11
   PV = 46%, NPV = 22%, BS = 16%, water = 2%, unclassified = 11%
2012-12
   PV = 47%, NPV = 20%, BS = 13%, water = 3%, unclassified = 14%
2013-01
    No data for 2013-01 , skipping...
2013-02
    No data for 2013-02 , skipping...
2013-03
   PV = 59%, NPV = 18%, BS = 17%, water = 3%, unclassified = 1%
2013-04
   PV = 57%, NPV = 18%, BS = 18%, water = 3%, unclassified = 0%
2013-05
   PV = 49%, NPV = 15%, BS = 20%, water = 2%, unclassified = 11%
2013-06
   PV = 61%, NPV = 21%, BS = 15%, water = 0%, unclassified

2015-10
   PV = 44%, NPV = 20%, BS = 32%, water = 0%, unclassified = 1%
2015-11
   PV = 42%, NPV = 15%, BS = 29%, water = 0%, unclassified = 12%
2015-12
   PV = 47%, NPV = 17%, BS = 33%, water = 0%, unclassified = 1%
2016-01
   PV = 46%, NPV = 13%, BS = 37%, water = 0%, unclassified = 0%
2016-02
   PV = 47%, NPV = 16%, BS = 34%, water = 0%, unclassified = 1%
2016-03
   PV = 45%, NPV = 20%, BS = 32%, water = 0%, unclassified = 1%
2016-04
   PV = 25%, NPV = 8%, BS = 22%, water = 0%, unclassified = 43%
2016-05
   PV = 46%, NPV = 16%, BS = 35%, water = 0%, unclassified = 0%
2016-06
   PV = 44%, NPV = 14%, BS = 40%, water = 0%, unclassified = 1%
2016-07
   PV = 30%, NPV = 13%, BS = 15%, water = 0%, unclassified = 40%
2016-08
   PV = 42%, NPV = 19%, BS = 36%, water = 0%, unclassified = 1%
2016-09
   PV = 44%, NPV = 20%, BS = 32%, water = 0%, unclassified = 1%
2016-10
   PV = 46%, NPV = 15%, BS = 35%, water = 0%, unclassified = 0%
2016-11
   PV = 47%, NPV = 17%, BS = 33%, water = 0%, unclassi

2013-02
   PV = 0%, NPV = 0%, BS = 0%, water = 0%, unclassified = 98%
2013-03
   PV = 84%, NPV = 11%, BS = 3%, water = 0%, unclassified = 1%
2013-04
   PV = 78%, NPV = 14%, BS = 6%, water = 0%, unclassified = 1%
2013-05
   PV = 67%, NPV = 11%, BS = 3%, water = 0%, unclassified = 17%
2013-06
   PV = 82%, NPV = 11%, BS = 2%, water = 0%, unclassified = 3%
2013-07
   PV = 78%, NPV = 13%, BS = 4%, water = 0%, unclassified = 2%
2013-08
   PV = 80%, NPV = 15%, BS = 2%, water = 0%, unclassified = 1%
2013-09
   PV = 79%, NPV = 16%, BS = 2%, water = 0%, unclassified = 1%
2013-10
   PV = 77%, NPV = 18%, BS = 2%, water = 0%, unclassified = 1%
2013-11
   PV = 78%, NPV = 18%, BS = 2%, water = 0%, unclassified = 1%
2013-12
   PV = 76%, NPV = 18%, BS = 3%, water = 0%, unclassified = 1%
2014-01
   PV = 71%, NPV = 12%, BS = 2%, water = 0%, unclassified = 13%
2014-02
   PV = 41%, NPV = 7%, BS = 4%, water = 0%, unclassified = 46%
2014-03
   PV = 81%, NPV = 12%, BS = 4%, water = 0%, unclassified = 1%
2014-

2016-08
   PV = 50%, NPV = 19%, BS = 27%, water = 0%, unclassified = 1%
2016-09
   PV = 51%, NPV = 19%, BS = 26%, water = 0%, unclassified = 1%
2016-10
   PV = 52%, NPV = 17%, BS = 28%, water = 0%, unclassified = 1%
2016-11
   PV = 53%, NPV = 17%, BS = 26%, water = 0%, unclassified = 1%
2016-12
   PV = 46%, NPV = 18%, BS = 20%, water = 0%, unclassified = 14%
Calculation complete for annual values
All data loaded, save to csv
SA2 102021053 done
SA2 ID: 102021054, size: 10.5216km^2, time: ('2011', '2016')
2011-01
   PV = 56%, NPV = 23%, BS = 18%, water = 1%, unclassified = 1%
2011-02
   PV = 54%, NPV = 26%, BS = 16%, water = 1%, unclassified = 1%
2011-03
   PV = 55%, NPV = 25%, BS = 16%, water = 1%, unclassified = 1%
2011-04
   PV = 48%, NPV = 16%, BS = 23%, water = 1%, unclassified = 9%
2011-05
   PV = 48%, NPV = 16%, BS = 20%, water = 0%, unclassified = 13%
2011-06
   PV = 54%, NPV = 18%, BS = 25%, water = 0%, unclassified = 1%
2011-07
   PV = 6%, NPV = 6%, BS = 5%, water = 0%, unclass